In [1]:
import pandas as pd

In [2]:
import pickle as pkl

In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

In [6]:
with open('features_juli_with_support.pkl', 'rb') as file:
    train = pkl.load(file)
    

In [53]:
with open('test_features_juli_with_support.pkl', 'rb') as file:
    test = pkl.load(file)

# Con mis features

In [7]:
train.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'label', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [8]:
train.shape

(19414, 13)

In [9]:
X_train = train[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia','modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [12]:
param_test1 = {
       'n_estimators': range(200, 1550, 50)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring=two_scorer,iid=False,cv=3,  verbose=3)
gsearch1.fit(train[predictors],train[target])

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


R": %2.9f -0.013750277092887675
ROC_AUC: %2.9f 0.5221290330188398
R": %2.9f 0.01664286111820712
ROC_AUC: %2.9f 0.6273510989941669
[CV] ....... n_estimators=200, score=0.5221290330188398, total=14.0min
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.3min remaining:    0.0s


R": %2.9f -0.0011355274384581282
ROC_AUC: %2.9f 0.5233207404727318
R": %2.9f 0.017766339404822862
ROC_AUC: %2.9f 0.617241428079789
[CV] ....... n_estimators=200, score=0.5233207404727318, total=12.4min
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 27.2min remaining:    0.0s


R": %2.9f -0.12591312949540256
ROC_AUC: %2.9f 0.5595787689554769
R": %2.9f 0.01894937238982486
ROC_AUC: %2.9f 0.6100636578990425
[CV] ....... n_estimators=200, score=0.5595787689554769, total=13.0min
[CV] n_estimators=250 ................................................
R": %2.9f -0.01397220986814074
ROC_AUC: %2.9f 0.5220057182994896
R": %2.9f 0.01818435616722236
ROC_AUC: %2.9f 0.6331320728418213
[CV] ....... n_estimators=250, score=0.5220057182994896, total=16.4min
[CV] n_estimators=250 ................................................
R": %2.9f -0.0014524167198533
ROC_AUC: %2.9f 0.5230011110769569
R": %2.9f 0.01932668840571694
ROC_AUC: %2.9f 0.6225795952830814
[CV] ....... n_estimators=250, score=0.5230011110769569, total=16.3min
[CV] n_estimators=250 ................................................
R": %2.9f -0.12692507914107876
ROC_AUC: %2.9f 0.5597326677741511
R": %2.9f 0.02032149033136721
ROC_AUC: %2.9f 0.6143682560672575
[CV] ....... n_estimators=250, score=0.5597326677741511, to

KeyboardInterrupt: 

In [10]:
y_train = train.label

In [19]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X_train,y_train,test_size=0.2)

In [20]:
model = ExtraTreesClassifier(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)



In [25]:
model.fit(Xtrn,Ytrn)
pred= model.predict_proba(Xtest)


In [26]:
pred.shape

(3883, 2)

In [27]:
Ytest.shape

(3883,)

In [30]:
roc_auc_score(Ytest, pred[:,1])

0.6358333664902718

In [31]:
help(ExtraTreesClassifier)

Help on class ExtraTreesClassifier in module sklearn.ensemble.forest:

class ExtraTreesClassifier(ForestClassifier)
 |  An extra-trees classifier.
 |  
 |  This class implements a meta estimator that fits a number of
 |  randomized decision trees (a.k.a. extra-trees) on various sub-samples
 |  of the dataset and uses averaging to improve the predictive accuracy
 |  and control over-fitting.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |      .. versionchanged:: 0.20
 |         The default value of ``n_estimators`` will change from 10 in
 |         version 0.20 to 100 in version 0.22.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  max_depth : integer or None, optional (default

In [33]:
from sklearn.model_selection import GridSearchCV

In [48]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid.fit(Xtrn, Ytrn)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [49]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [50]:
grid.best_score_

0.9499710256905544

In [52]:
roc_auc_score(Ytest, grid.predict_proba(Xtest)[:,1])

0.6685247814957758

In [55]:
test.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [56]:
x_test = test[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [75]:
sum(pd.isnull(x_test['event_support']))

0

In [66]:
x_test['viewed product'].fillna(x_test['viewed product'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [60]:
x_test['visited site'].fillna(x_test['visited site'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [62]:
x_test['generic listing'].fillna(x_test['generic listing'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [64]:
x_test['conversion'].fillna(x_test['conversion'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [79]:
pred_porta = grid.predict_proba(x_test)[:,1]

In [77]:
to_puvlic = pd.DataFrame()

In [78]:
to_puvlic['person'] = test.person

In [80]:
to_puvlic['label'] = pred_porta

In [82]:
to_puvlic.to_csv('extraTreesJuli.csv', index = False)

# Con features de las chicas

In [84]:
with open('features_chicas.pkl','rb') as file:
    features = pkl.load(file)

In [85]:
features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [96]:
features['device_frecuente'].fillna(features['device_frecuente'].mode()[0], inplace=True)

In [97]:
features[ 'Returning'].fillna(features['Returning'].mode()[0], inplace=True)

In [98]:
import category_encoders as ce

In [99]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [89]:
label_train = pd.read_csv('data/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [90]:
train_features = pd.merge(features, label_train, on='person', how='right')

In [101]:
train_features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'label'],
      dtype='object')

In [125]:
sum(pd.isnull(train_features['device_frecuente']))

288

In [121]:
train_features['Returning'].fillna(train_features['Returning'].mode()[0], inplace=True)

In [126]:
train_features['device_frecuente'].fillna(train_features['device_frecuente'].mode()[0], inplace=True)

In [127]:
x_train_features = train_features[[ 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [103]:
y_train_features = train_features.label

In [128]:
x_train_features.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(x_train_features.channel_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [129]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [130]:
x_train_features.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(x_train_features.device_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [131]:
x_train_features.shape

(19414, 27)

In [132]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn2, Xtest2, Ytrn2, Ytest2 = train_test_split(x_train_features,y_train_features,test_size=0.2)

In [133]:
param_grid2 = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid2 = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid2.fit(Xtrn2, Ytrn2)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  8.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [134]:
grid2.best_score_

0.9490696027300238

In [138]:
pred = grid2.predict_proba(Xtest2)[:,1]

In [139]:
roc_auc_score(Ytest2,pred)

0.7785501597333149

In [135]:
test_person = pd.read_csv('../7506_ml/data/trocafone_kaggle_test.csv')

In [136]:
test_person.shape

(19415, 1)

In [137]:
test2 = pd.merge(features , test_person, on='person', how='right')

In [142]:
test2.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(test2.channel_frecuente))

In [143]:
test2.device_frecuente = te_device_frecuente.transform(pd.DataFrame(test2.device_frecuente))

In [145]:
test2.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [146]:
xtest2 = test2[['cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [168]:
sum(pd.isnull(xtest2['device_frecuente']))

0

In [167]:
xtest2['channel_frecuente'].fillna(xtest2['channel_frecuente'].mode()[0], inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [170]:
pred = grid2.predict_proba(xtest2)[:,1]

In [171]:
to_publish = pd.DataFrame()

In [172]:
to_publish['person'] = test2.person

In [173]:
to_publish['label'] = pred

In [175]:
to_publish.to_csv('extraTreesJuli2.csv', index=False)